<a href="https://colab.research.google.com/github/tailagos/computervisionmodel/blob/main/Remote_Sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import ee
import geemap

In [2]:
ee.Authenticate()

In [4]:
ee.Initialize(project= 'ee-abelekeleattai')

In [6]:
a = 2
print (type (a))
sum = a+1
print (sum)
sum == 3

<class 'int'>
3


True

In [7]:
b = ee.Number(2)
print (type(b))
sum = b.add(1)
print (sum)

<class 'ee.ee_number.Number'>
ee.Number({
  "functionInvocationValue": {
    "functionName": "Number.add",
    "arguments": {
      "left": {
        "constantValue": 2
      },
      "right": {
        "constantValue": 1
      }
    }
  }
})


In [8]:
b == 2

False

In [9]:
b.eq(2)

In [10]:
print(a+b)

TypeError: unsupported operand type(s) for +: 'int' and 'Number'

In [11]:
print(a.add(b))

AttributeError: 'int' object has no attribute 'add'

In [13]:
b.getInfo()

2

In [14]:
bb = b.getInfo()

In [15]:
print(a+bb)

4


In [16]:
ROI = ee.Geometry.Polygon(
[[67.42,26.54],
[68.65,26.54],
[68.65,27.73],
[67.42,27.73]]
)

In [17]:
pre_flood_start_date = '2021-08-01'
pre_flood_end_date = '2021-09-30'
flood_start_date = '2022-08-01'
flood_end_date = '2022-09-30'

In [19]:
m = geemap.Map()
m.addLayer(ROI, {'color':'green'}, 'Region of Interest')
m.centerObject(ROI,9)
m

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:
S1Collection = ee.ImageCollection('COPERNICUS/S1_GRD')

In [23]:
S1Filtered = (
    S1Collection
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
'VH'))
    .filter(ee.Filter.eq('orbitProperties_pass','ASCENDING'))
    .filterBounds(ROI)
)

print('Number of images: ',S1Filtered.size().getInfo())

Number of images:  987


In [25]:
S1Filtered2021=(
    S1Filtered
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .select(['VV','VH'])
)
S1Filtered2022=(
    S1Filtered
    .filterDate(flood_start_date, flood_end_date)
    .select(['VV','VH'])
)
# let's find out how many satellite images we have got on each image collection
print('Number of pre flood images: ',S1Filtered2021.size().getInfo())
print('Number of post flood images: ',S1Filtered2022.size().getInfo())

Number of pre flood images:  20
Number of post flood images:  20


In [26]:
beforeImage = S1Filtered2021.mean()
duringImage = S1Filtered2022.mean()
# by using ".mean()" we convert a collection of images into a single image. The
# value of each pixel in the new image is equal to the mean value calculated
# from the image collection on a pixel by pixel basis

print(type(S1Filtered2021))
# this evaluates to an ee.imagecollection

print(type(beforeImage))
# this evaluates to an ee.image

<class 'ee.imagecollection.ImageCollection'>
<class 'ee.image.Image'>


In [27]:
m = geemap.Map()
m.addLayer(beforeImage,{},
'before mean image')

m.addLayer(duringImage,{},'during mean image')
m.addLayer(ROI,{'color':'green'},'ROI')
m.centerObject(ROI,9)
m

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [28]:
beforeImage = beforeImage.clip(ROI)
duringImage = duringImage.clip(ROI)
m = geemap.Map()
m.addLayer(beforeImage,{},
'before mean image')
m.addLayer(duringImage,{},
'during mean image')
m.centerObject(ROI,9)
m

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [29]:
beforeImage = beforeImage.addBands(
    beforeImage.select('VV').divide(beforeImage.select('VH')).rename('VV/VH'))
duringImage = duringImage.addBands(duringImage.select('VV').divide(
    duringImage.select('VH')).rename('VV/VH'))
visParams = {'min':[-25,-25,0],
             'max':[0,0,2]
}

In [30]:
m = geemap.Map()
m.centerObject(ROI,9)
m.addLayer(beforeImage,visParams,'before mean image')
m.addLayer(duringImage,visParams,'during mean image')
m

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [31]:
left_layer = geemap.ee_tile_layer(beforeImage, visParams,'before the flood')
right_layer = geemap.ee_tile_layer(duringImage, visParams,'during the flood')
m.split_map(
    left_layer, right_layer, left_label='Before', right_label='During')
m

Map(bottom=55565.0, center=[27.135283876202667, 68.03500000000001], controls=(ZoomControl(options=['position',…

In [32]:
AOI = ee.Geometry.Polygon([[-0.79,53.75],[-0.79,53.55],[0.16,53.55],[0.16,53.75]])
m = geemap.Map()
m.addLayer(AOI,{},'Humber area')
m.centerObject(AOI)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
# Imports the global surface water dataset
gsw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
Humber = (gsw
          .clip(AOI) # Clip to the Area of Interest
          .select('seasonality').gte(12)
          .selfMask()
)
m = geemap.Map()
m.addLayer(Humber,{},'Humber estuary')
m.centerObject(AOI)
m

Humber = Humber.reduceToVectors()
m = geemap.Map()
m.addLayer(Humber,{},'Humber estuary')
m.centerObject(AOI)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [34]:
S2Collection= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [35]:
S2CollectionA = (
    S2Collection
    .filterDate('2021-04-04', '2021-04-05')
    .filterBounds(Humber)
)
S2CollectionB = (
    S2Collection
    .filterDate('2021-04-19', '2021-04-20')
    .filterBounds(Humber)
)
print(S2CollectionA.size().getInfo())
print(S2CollectionB.size().getInfo())

3
3


In [36]:
S2ImageA = S2CollectionA.first().clip(Humber)
S2ImageB = S2CollectionB.first().clip(Humber)

visualization = {'bands':['B4','B3','B2'],
                 'min':146,
                 'max':2183,
                 'gamma':1.4}
m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(S2ImageA,
                                  visualization,
                                  '4 April 2021')

right_layer = geemap.ee_tile_layer(S2ImageB,
                                   visualization,
                                   '19 April 2021')
m.split_map(
    left_layer, right_layer,
    left_label='4 April 2021', right_label= '19 April 2021')
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [40]:
ndtiA = S2ImageA.normalizedDifference(['B4','B3']).rename('NDTI')

ndtiB = S2ImageB.normalizedDifference(['B4','B3']).rename('NDTI')

visParams = {'min':-0.15, 'max':0.15, 'palette':['lightblue','white','brown']}

m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(ndtiA, visParams,'NDTI A')
right_layer = geemap.ee_tile_layer(ndtiB,visParams,'NDTI B')

m.split_map(
    left_layer, right_layer, left_label='4 April 2021', right_label='19 April 2021')

m.add_colorbar(
    vis_params=visParams,
    label='NDTI',
    position='bottomright'
)
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [43]:
ndtiA = S2ImageA.normalizedDifference(['B5','B4']).rename('NDCI')

ndtiB = S2ImageB.normalizedDifference(['B5','B4']).rename('NDCI')

visParams = {'min':-0.15, 'max':0.15, 'palette':['lightblue','white','brown']}

m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(ndtiA, visParams,'NDCI A')
right_layer = geemap.ee_tile_layer(ndtiB,visParams,'NDCI B')

m.split_map(
    left_layer, right_layer, left_label='4 April 2021', right_label='19 April 2021')

m.add_colorbar(
    vis_params=visParams,
    label='NDCI',
    position='bottomright'
)
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…